**<h1 align="center">PROYECTO INDIVIDUAL 1</h1>**
<h1 align="center"> Merge API</h1>

In [10]:
import pandas as pd
import fsspec
import pyarrow.parquet as pq

In [11]:
#Abrimos los archivos parquet
user_games = pd.read_parquet('DatasetsLimpios\\games.parquet')
user_items = pd.read_parquet('DatasetsLimpios\\user_items.parquet')
user_reviews = pd.read_parquet('DatasetsLimpios\\user_reviews.parquet')

In [12]:
#Seleccionamos las columnas relevantes de "user_items"
user_items = user_items[['user_id','item_id','playtime_forever']]
#Generamos un identificador único en el Dataframe "user_items"
user_items['item_id'] = user_items['item_id'].astype(str)
user_items['id'] = user_items['user_id'] + user_items['item_id']
#Renombramos la columna "id" por "item_id" de la tabla "user_games"
user_games = user_games.rename(columns={'id': 'item_id'})
#Cambiamos el tipo de dato de la columna 'item_id' del Dataframe "user_games"
user_games['item_id'] = user_games['item_id'].astype(str)
#Generamos un identificador único en el Dataframe "user_reviews"
user_reviews['item_id'] = user_reviews['item_id'].astype(str)
user_reviews['id'] = user_reviews['user_id'] + user_reviews['item_id']

In [13]:
#Unimos los dataframes "user_reviews" y "user_games" por el 'item_id' y eliminacion de nulos
merged_df = user_reviews.merge(user_games, on='item_id', how='left')
merged_df.dropna(inplace=True)
#Generamos dataframe definitivo uniendo 'items' con 'merge_df' por el identificador unico 'id' de nombre "steam"
steam = user_items.merge(merged_df, on='id')
#Renombramos columnas del dataframe 'steam'
steam = steam.rename(columns={'user_id_x':'user_id'})
steam = steam.rename(columns={'item_id_x':'item_id'})
#Eliminamos las columnas redundantes del dataframe 'steam'
steam.drop('user_id_y', axis='columns', inplace=True)
steam.drop('item_id_y', axis='columns', inplace=True)

In [14]:
#Cambiamos los tipos de datos de algunas columnas
steam['year_posted'] = steam['year_posted'].astype('int64')
steam['playtime_forever'] = steam['playtime_forever'].astype('int64')
steam['year'] = steam['year'].astype('int64')

In [15]:
#Seleccionamos las columnas definitivas
steam = steam[['user_id','item_id','playtime_forever','recommend','review','year_posted','genres','title','developer','year']]

In [16]:
#Guardamos archivo en .parquet
steam.to_parquet('Merge\\Tabla_API.parquet')